In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2 
from skimage.feature import local_binary_pattern
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import euclidean_distances


In [3]:
# load images of ORL
Images = []
Labels = []
files = glob.glob("./ORL/*.jpg")
files.sort()
for img in files:
    tile= cv2.imread(img)
    tile = cv2.cvtColor(tile, cv2.COLOR_BGR2GRAY)
    label = img.split("_")[1].split(".")[0]
    Labels.append(int(label))
    Images.append(tile)
    
Images = np.array(Images)
Labels = np.array(Labels)
print("Shapes of Dataset:",Images.shape)


Shapes of Dataset: (410, 80, 70)


In [3]:
def BinaryPixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass

    return new_value

def LBPPixelCalculator(img, x, y):

    center = img[x][y]

    val_ar = []

    # top_left
    val_ar.append(BinaryPixel(img, center, x-1, y-1))

    # top
    val_ar.append(BinaryPixel(img, center, x-1, y))

    # top_right
    val_ar.append(BinaryPixel(img, center, x-1, y + 1))

    # right
    val_ar.append(BinaryPixel(img, center, x, y + 1))

    # bottom_right
    val_ar.append(BinaryPixel(img, center, x + 1, y + 1))

    # bottom
    val_ar.append(BinaryPixel(img, center, x + 1, y))

    # bottom_left
    val_ar.append(BinaryPixel(img, center, x + 1, y-1))

    # left
    val_ar.append(BinaryPixel(img, center, x, y-1))

    # Now, we need to convert binary
    # values to decimal
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]

    val = 0

    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]

    return val


def GetLBPFeatures(img_bgr):
    height, width = img_bgr.shape
    img_lbp = np.zeros((height, width),np.uint8)
    for i in range(0, height):
        for j in range(0, width):
            img_lbp[i, j] = LBPPixelCalculator(img_bgr, i, j)
            
    hist = plt.hist(np.reshape(img_lbp,(width*height,)),bins=256)
    plt.clf()
    
    #normal 
    hist = hist[0]/sum(hist[0])
    
    return hist
    



In [4]:
X_train = []

for i in range(len(Images)):
    X_train.append(GetLBPFeatures(Images[i]))
X_train = np.array(X_train)
X_train.shape

(410, 256)

<Figure size 432x288 with 0 Axes>

In [5]:

index = 0
OutPuts = []
while index !=200:
    index+=1
    X_train_, X_test, Y_train, Y_test = train_test_split(X_train, Labels, test_size=0.2, stratify=Labels)

    knn = KNeighborsClassifier(n_neighbors=3,metric="cosine")
    knn.fit(X_train_,Y_train)
    OutPuts.append (knn.score(X_test,Y_test))

In [6]:
print("Mean of accuracy for Actual features:",np.mean(OutPuts))

Mean of accuracy for Actual features: 0.853719512195122


In [7]:
while abs(np.mean(OutPuts) - knn.score(X_test,Y_test)) >= 0.01:
    X_train_, X_test, Y_train, Y_test = train_test_split(X_train, Labels, test_size=0.25, stratify=Labels)
    knn = KNeighborsClassifier(n_neighbors=3,metric="cosine")
    knn.fit(X_train_,Y_train)

In [8]:
knn.score(X_test,Y_test)

0.8536585365853658

In [9]:
with open('X-Train.npy', 'wb') as f:
    np.save(f, X_train_)
    
with open('Y-Train.npy', 'wb') as f:
    np.save(f, Y_train)
    
with open('X-Test.npy', 'wb') as f:
    np.save(f, X_test)
    
with open('Y-Test.npy', 'wb') as f:
    np.save(f, Y_test)
    